## Preprocessing

In [1]:
# Import our dependencies 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
import pandas as pd 
import tensorflow as tf 
import numpy as np 

# Import pandas and read the charity_data.csv from the provided cloud URL. 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv") 
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'. 
application_df = application_df.drop(columns=['EIN', 'NAME']) 
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column. 
unique_values = application_df.nunique() 
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other" 
application_type_counts = application_df['APPLICATION_TYPE'].value_counts() 
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced 
cutoff = 100 # Set cutoff for less frequent values 
application_types_to_replace = application_type_counts[application_type_counts < cutoff].index.tolist() 

# Replace in dataframe 
for app in application_types_to_replace: 
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other") 

# Check to make sure replacement was successful 
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other" 
classification_counts = application_df['CLASSIFICATION'].value_counts() 
print(classification_counts.head(10))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
Name: count, dtype: int64


In [7]:
# Filter classifications with more than 1 occurrence
classification_counts_filtered = classification_counts[classification_counts > 1] 
print(classification_counts_filtered)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced 
cutoff_value = 50 
classifications_to_replace = classification_counts[classification_counts <= cutoff_value].index.tolist() 

# Replace in dataframe 
for cls in classifications_to_replace: 
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other") 

# Check to make sure replacement was successful 
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      441
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with pd.get_dummies 
application_df_dummies = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays 
X = application_df_dummies.drop("IS_SUCCESSFUL", axis=1) 
y = application_df_dummies["IS_SUCCESSFUL"] 

# Split the preprocessed data into a training and testing dataset 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Create a StandardScaler instance 
scaler = StandardScaler() 

# Fit the StandardScaler 
X_scaler = scaler.fit(X_train) 

# Scale the data 
X_train_scaled = X_scaler.transform(X_train) 
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [24]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import LeakyReLU

# Learning rate decay function (slower decay rate)
def lr_decay(epoch, lr):
    decay_rate = 0.98  # Slower decay rate
    return lr * decay_rate

# Learning rate decay scheduler callback
lr_decay_callback = LearningRateScheduler(lr_decay)

# Redefine and compile the model
nn = tf.keras.models.Sequential()

# Adding layers with regularization and dropout
nn.add(tf.keras.layers.Dense(units=128, input_dim=X_train_scaled.shape[1]))
nn.add(LeakyReLU())  # Use LeakyReLU as a layer, not as a string
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))  # Reduced dropout rate

nn.add(tf.keras.layers.Dense(units=64))
nn.add(LeakyReLU())  # Use LeakyReLU as a layer, not as a string
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))  # Reduced dropout rate

nn.add(tf.keras.layers.Dense(units=32))
nn.add(LeakyReLU())  # Use LeakyReLU as a layer, not as a string
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))  # Reduced dropout rate

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

C:\Users\josel\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           6,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,177 (71.00 KB)

 Trainable params: 17,729 (69.25 KB)

 Non-trainable params: 448 (1.75 KB)

In [26]:
# Learning rate scheduler callback
def lr_scheduler(epoch, lr):
    if epoch < 100:
        return lr
    else:
        return lr * 0.9  # Decrease learning rate by 10% after epoch 100

lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

# Compile the model with the adjusted learning rate decay callback
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
import numpy as np

# Convert to float32 for X and int32 for y
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)

# Check shapes and data types
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Optional: Check for NaN or Inf values
print("NaN in X_train?", np.any(np.isnan(X_train)))
print("NaN in X_test?", np.any(np.isnan(X_test)))
print("NaN in y_train?", np.any(np.isnan(y_train)))
print("NaN in y_test?", np.any(np.isnan(y_test)))

# Train the model with the learning rate scheduler callback
history = nn.fit(X_train_scaled, y_train, epochs=150, batch_size=64, 
                  validation_data=(X_test_scaled, y_test), 
                  callbacks=[lr_decay_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

X_train shape: (25724, 53)
X_test shape: (8575, 53)
y_train shape: (25724,)
y_test shape: (8575,)
NaN in X_train? False
NaN in X_test? False
NaN in y_train? False
NaN in y_test? False
Epoch 1/150
402/402 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.6655 - loss: 0.6659 - val_accuracy: 0.7289 - val_loss: 0.5665 - learning_rate: 9.8000e-04
Epoch 2/150
402/402 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7132 - loss: 0.5876 - val_accuracy: 0.7301 - val_loss: 0.5593 - learning_rate: 9.6040e-04
Epoch 3/150
402/402 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7174 - loss: 0.5756 - val_accuracy: 0.7294 - val_loss: 0.5601 - learning_rate: 9.4119e-04
Epoch 4/150
402/402 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7219 - loss: 0.5695 - val_accuracy: 0.7305 - val_loss: 0.5581 - learning_rate: 9.2237e-04
Epoch 5/150
402/402 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7295 - loss: 0.5594 - val_accuracy: 0.7266 - val_loss: 0.5585 - learning_rate: 9.0392e-04
Epoch 6/150
402/402 ━━━━━━━━━━━

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7324 - loss: 0.5477
Loss: 0.5477253198623657, Accuracy: 0.7323614954948425


In [35]:
# Save the optimized model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')
print("Model has been saved to 'AlphabetSoupCharity_Optimization.h5'.")

Model has been saved to 'AlphabetSoupCharity_Optimization.h5'.
